<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/eceg_cipher_point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import random
import sympy as sp
import json
import base64

class EllipticCurveElGamal:
  def __init__(self, a, b, p, k, B=None):
    self.p = p # primer number
    self.a = a # alpha
    self.b = b # beta

    self.k = k
    self.PointB = B

    self.pointP = None
    self.pointQ = None


    self.char_to_num_dict = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
    'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17,
    'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25,
    'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33,
    'y': 34, 'z': 35, '.': 36, '#': 37, '*': 38, '/': 39, '-': 40
    }

    self.num_to_char_dict = {v: k for k, v in self.char_to_num_dict.items()}

    # self.k_koblitz = max(self.char_to_num_dict.values()) + 1  # max 24 for 35 char
    self.k_koblitz = 18  # max 24 for 35 char

  def example_function(self):
      print(f"Using persistent k value: {self.k}")

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  def is_on_curve(self, x, y):
    return self.elliptic_curve_equation(x)  == (y**2) % self.p

  # def generate_random_number(self):
  #   self.k = random.randint(1, self.p - 1)
  #   return self.k

  def generate_random_point(self):
    while True:
      x = random.randint(1, self.p - 1)
      y = random.randint(1, self.p - 1)
      if self.is_on_curve(x, y):
        return (x, y)

  def mod_inverse(self, a, p):
      if a == 0:
          raise ValueError("Inverse does not exist for 0.")
      return pow(a, p - 2, p)

  def calculate_slope_mod(self, P, Q):
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 == y2:
        # Use the formula for point doubling when P == Q
        numerator = (3 * x1**2 + self.a) % self.p
        denominator = (2 * y1) % self.p
    else:
        # Use the formula for regular slope when P != Q
        numerator = (y2 - y1) % self.p
        denominator = (x2 - x1) % self.p

    if denominator == 0:
        raise ValueError("Slope is undefined (denominator is zero).")

    # Compute the slope as (numerator / denominator) % p, which is
    # numerator * mod_inverse(denominator, p) % p
    slope = (numerator * self.mod_inverse(denominator, self.p)) % self.p
    return slope

  def calculate_add_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - x1 - x2) % self.p
    return xr

  def calculate_add_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    x2, y2 = Q
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_addition(self, P, Q):
    m = self.calculate_slope_mod(P, Q)

    xr = self.calculate_add_xr_mod(P, Q, m)

    yr = self.calculate_add_yr_mod(P, Q, m, xr)

    R = (xr, yr)

    return R

  def calculate_dob_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - (2 * x1)) % self.p
    return xr

  def calculate_dob_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_doubling(self, P):

    m = self.calculate_slope_mod(P, P)

    xr = self.calculate_dob_xr_mod(P, P, m)

    yr = self.calculate_dob_yr_mod(P, P, m, xr)

    R = (xr, yr)

    return R

  def calculate_point_multiplication(self, P, k):
    if k == 0:
      return None
    elif k == 1:
      return P

    # Initialize R to be the point at infinity, often represented as None in this context
    R = None
    Q = P  # Start with Q as P

    # Double-and-add method
    while k > 0:
        if k % 2 == 1:  # If k is odd, add Q to the result
            if R is None:
                R = Q  # R is the point at infinity initially
            else:
                R = self.calculate_point_addition(R, Q)
        Q = self.calculate_point_addition(Q, Q)  # Double the point Q
        k //= 2  # Move to the next bit

    return R

  def calculate_point_subtract(self, P, Q):
    # Find the inverse of point Q (x_Q, y_Q) -> (x_Q, -y_Q mod p)
    Q_inv = (Q[0], (-Q[1]) % self.p)

    # Subtract P - Q by adding P and Q_inv
    R = self.calculate_point_addition(P, Q_inv)

    return R

  def generate_public_key(self, private_key):
    public_key = self.calculate_point_multiplication(self.PointB, private_key)
    return public_key

  def encryption(self, M, public_key):
    max_attempts = 100  # Limit the number of retries for generating valid `k`
    attempts = 0

    while attempts < max_attempts:
        k = random.randint(1, self.p - 1)  # Generate random `k`, 1 < k < p - 1

        # Calculate C1 = k * B (public generator point)
        C1 = self.calculate_point_multiplication(self.PointB, k)

        # If C1 is not valid or not on the curve, retry
        if C1 is None or not self.is_on_curve(*C1):
            attempts += 1
            continue

        # Calculate C2 = M + k * public_key
        k_times_public_key = self.calculate_point_multiplication(public_key, k)
        if k_times_public_key is None or not self.is_on_curve(*k_times_public_key):
            attempts += 1
            continue

        C2 = self.calculate_point_addition(M, k_times_public_key)

        # If C2 is not valid or not on the curve, retry
        if C2 is None or not self.is_on_curve(*C2):
            attempts += 1
            continue

        # If both C1 and C2 are valid, return the encrypted pair (C1, C2)
        return (C1, C2)

    # If no valid (C1, C2) pair is found after max_attempts, raise an error
    raise ValueError(f"Failed to find valid encryption after {max_attempts} attempts")



  def decryption(self, C, private_key):
      C1, C2 = C

      # Calculate _p = private_key * C1
      _p = self.calculate_point_multiplication(C1, private_key)

      # Calculate the plain point = C2 - _p
      plain = self.calculate_point_subtract(C2, _p)

      # Verify the result is on the curve
      if not self.is_on_curve(*plain):
          raise ValueError(f"Decrypted point {plain} is not on the curve")

      return plain


  # Function to map char to num
  def char_to_num(self, char):
      # Ensure the input is a string; if not, convert it
      char = str(char)

      # Check if the string exists in the dictionary
      if char in self.char_to_num_dict:
          return self.char_to_num_dict[char]
      else:
          # Handle the expanded character set (0-40)
          raise ValueError(f"Character '{char}' is not valid. Please use '0-9', 'a-z', or other valid characters.")


  # Function to map num to char
  def num_to_char(self, num):
      # Ensure that the input is an integer; if it's a string digit, convert to integer
      if isinstance(num, str) and num.isdigit():
          num = int(num)

      # Now we expect num to be an integer
      if isinstance(num, int):
          if num in self.num_to_char_dict:  # Handle numbers 0-40 by converting them using the dictionary
              return self.num_to_char_dict[num]
          else:
              raise ValueError(f"Number '{num}' is out of the valid range (0-40).")
      else:
          raise ValueError(f"Input '{num}' is not valid. Please provide a valid number.")


  def message_to_koblitz(self, message):
      # Convert the message to a list of characters
      chars = list(message)

      koblitz = []
      for char in chars:
          num = self.char_to_num(char)
          koblitz.append(num)

      return koblitz

  def koblitz_to_message(self, koblitz):
      message = []
      for num in koblitz:
          char = self.num_to_char(num)
          message.append(char)

      return ''.join(message)

  def koblitz_encode(self, m, max_attempts=1000):
    num = 1  # Start with x = m * k + 1
    attempts = 0

    while attempts < max_attempts:
        x = m * self.k_koblitz + num
        rhs = (x**3 + self.a * x + self.b) % self.p  # right-hand side of the elliptic curve equation

        # Check if rhs is a quadratic residue modulo p
        if sp.is_quad_residue(rhs, self.p):
            y = sp.sqrt_mod(rhs, self.p)
            return (x, y)  # Return the point (x, y) as a tuple

        num += 1  # Increment to check next x value
        attempts += 1

    # If no valid point is found after max_attempts
    raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

  def koblitz_encode_message(self, message):
    encoded_points = []
    for char in message:
      encoded_point = self.koblitz_encode(char)
      encoded_points.append(encoded_point)
      # print(f"Encoded point for character '{char}': {encoded_point}")

    return encoded_points

  def koblitz_decode(self, x):
    return (x - 1) // self.k_koblitz

  def koblitz_decode_message(self, points):
      decoded_message = []
      for point in points:
          x, y = point  # You can access both x and y here
          decoded_num = self.koblitz_decode(x)  # Decode based on x-coordinate
          decoded_message.append(decoded_num)
      return decoded_message

  def encrypt_message(self, message, public_key):
      # Encode the message into Koblitz points
      koblitz_message = self.message_to_koblitz(message)
      encoded_message = self.koblitz_encode_message(koblitz_message)
      encrypted_points = []

      # Encrypt each Koblitz point
      for point in encoded_message:
          encrypted_point = self.encryption(point, public_key)
          encrypted_points.append(encrypted_point)
      print(encrypted_points)
      # Convert encrypted_points (a list of tuples) to a JSON string
      encrypted_message_string = json.dumps(encrypted_points)

      # Optionally, encode the string to base64 for a cleaner ciphertext
      encrypted_message_base64 = base64.b64encode(encrypted_message_string.encode()).decode()

      return encrypted_message_base64, encrypted_points  # Return as readable string

  def decrypt_message(self, encrypted_message_base64, private_key):
      # Decode the base64 string back to the JSON string
      encrypted_message_string = base64.b64decode(encrypted_message_base64).decode()

      # Convert the JSON string back to a list of encrypted points (tuples)
      encrypted_points = json.loads(encrypted_message_string)

      decrypted_message = []

      # Decrypt each point
      for point in encrypted_points:
          decrypted_point = self.decryption(point, private_key)
          decrypted_message.append(decrypted_point)

      # Decode the Koblitz points into the original numeric values
      decoded_koblitz_points = self.koblitz_decode_message(decrypted_message)

      # Convert numeric Koblitz points to characters using num_to_char
      plain_text_message = ''.join([self.num_to_char(num) for num in decoded_koblitz_points])

      return plain_text_message



In [54]:
elliptic = EllipticCurveElGamal(a=1, b=1, p=1019, k=3)

base = elliptic.generate_random_point()
elliptic.PointB = base

base2 = elliptic.generate_random_point()

message = "hello"
private_key = 5
public_key = elliptic.generate_public_key(private_key=private_key)

# Encrypt the message
encrypted_message_string, encrypted_points = elliptic.encrypt_message(message, public_key)
print(f"Encrypted message: {encrypted_message_string}")
print(f"Encrypted points: {encrypted_points}")

# Decrypt the message
decrypted_message = elliptic.decrypt_message(encrypted_message_string, private_key)
print(f"Decrypted message: {decrypted_message}")

[((615, 768), (711, 498)), ((604, 623), (621, 250)), ((324, 408), (634, 538)), ((692, 550), (169, 387)), ((266, 921), (244, 610))]
Encrypted message: W1tbNjE1LCA3NjhdLCBbNzExLCA0OThdXSwgW1s2MDQsIDYyM10sIFs2MjEsIDI1MF1dLCBbWzMyNCwgNDA4XSwgWzYzNCwgNTM4XV0sIFtbNjkyLCA1NTBdLCBbMTY5LCAzODddXSwgW1syNjYsIDkyMV0sIFsyNDQsIDYxMF1dXQ==
Encrypted points: [((615, 768), (711, 498)), ((604, 623), (621, 250)), ((324, 408), (634, 538)), ((692, 550), (169, 387)), ((266, 921), (244, 610))]
Decrypted message: hello


In [55]:
elliptic.is_on_curve(114, 172)

True

In [56]:
nums = elliptic.koblitz_decode_message([(169, 632), (362, 28)])
nums

[9, 20]

In [57]:
point = elliptic.koblitz_encode_message(nums)
point

[(165, 312), (362, 28)]

In [58]:
nums = elliptic.koblitz_decode_message(point)
nums

[9, 20]

In [59]:
point = elliptic.koblitz_encode_message(nums)
point

[(165, 312), (362, 28)]

In [60]:
# Define a function to normalize and map points to a number in the 0-40 range
def map_to_range(value, min_val, max_val, target_min, target_max):
    # Normalize the value to the range 0-1
    normalized = (value - min_val) / (max_val - min_val)
    # Map to the target range
    return target_min + normalized * (target_max - target_min)

# Define a function to reverse the mapping from the 0-40 range back to the original range
def reverse_mapping(mapped_value, target_min, target_max, original_min, original_max):
    # Normalize the value back to the original range
    normalized = (mapped_value - target_min) / (target_max - target_min)
    return original_min + normalized * (original_max - original_min)

# Define the points
point1 = (169, 632)
point2 = (362, 28)

# Define the hypothetical min/max for the original x and y ranges
x_min, x_max = 0, 1000  # Assuming the original x can range from 0 to 1000
y_min, y_max = 0, 1000  # Assuming the original y can range from 0 to 1000

# Define the target range (0-40)
target_min, target_max = 0, 40

# Normalize and map the points to the 0-40 range
mapped_x1 = map_to_range(point1[0], x_min, x_max, target_min, target_max)
mapped_y1 = map_to_range(point1[1], y_min, y_max, target_min, target_max)
mapped_x2 = map_to_range(point2[0], x_min, x_max, target_min, target_max)
mapped_y2 = map_to_range(point2[1], y_min, y_max, target_min, target_max)

# Combine into single numbers for each point
mapped_point1 = (mapped_x1, mapped_y1)
mapped_point2 = (mapped_x2, mapped_y2)

# Reverse the mapping to get back the original points (within some precision)
restored_x1 = reverse_mapping(mapped_x1, target_min, target_max, x_min, x_max)
restored_y1 = reverse_mapping(mapped_y1, target_min, target_max, y_min, y_max)
restored_x2 = reverse_mapping(mapped_x2, target_min, target_max, x_min, x_max)
restored_y2 = reverse_mapping(mapped_y2, target_min, target_max, y_min, y_max)

restored_point1 = (restored_x1, restored_y1)
restored_point2 = (restored_x2, restored_y2)

mapped_point1, mapped_point2, restored_point1, restored_point2


((6.760000000000001, 25.28),
 (14.48, 1.12),
 (169.0, 632.0),
 (362.0, 28.000000000000004))